In [156]:
from analysis.analysis import *
import matplotlib.pyplot as plt
from glob import glob
import os
import numpy as np


def clean_init_pts(pts, labels, nuke):
    df = pd.DataFrame({"x": pts[:, 0], "y": pts[:, 1], "color": labels})
    if nuke:
        df = df.drop(df.query("y < 30 and y > -30 and color == 'red'").index)

    return df[df.y < 160].reset_index(drop=True)


def heatmap(
    ax,
    F,
    bounds,
    labels,
    end_pts,
    xlabel=False,
    ylabel=False,
    cbar=False,
    cbar_label=False,
    cmap=None,
):
    xmin, xmax, vmin, vmax, nbins = bounds
    xmin, xmax = np.array([xmin, xmax]) - (xmax + xmin) / 2  # center along x
    ax.set_xlabel(r"$x$ ($\mu$m)") if xlabel else ...
    ax.set_ylabel(r"$v$ ($\mu$m/hr)") if ylabel else ...
    _kwargs = {
        "extent": [xmin, xmax, vmin, vmax],
        "origin": "lower",
        "interpolation": "bilinear",
    }
    if cmap is None:
        cmap = "jet"
    im = ax.imshow(F, cmap=cmap, **_kwargs)
    if cbar:
        cbar = plt.colorbar(im, ax=ax)
        cbar.set_ticks([-1000, 0, 1000])
        cbar.formatter.set_powerlimits((0, 0))
        cbar.formatter.set_useMathText(True)
    cbar.set_label(r"$F$ ($\mu$m/hr$^2$)") if cbar_label else ...
    ax.set_aspect("auto")


def streamplot(
    F,
    bounds,
    labels,
    end_pts,
    ax,
    title="",
    n_skip=1,
    zero_line=False,
    show_terminal=None,
    septx=False,
    darker_shade=False,
    mix_corn_salmon=False,
    ignore_wheat=False,
    include_red=True,
    nuke_red_in_middle=True,
    bootstrap_flag=False,
    **kwargs,
):
    xmin, xmax, vmin, vmax, nbins = bounds
    x_c = (xmax + xmin) / 2
    xmin, xmax = np.array([xmin, xmax]) - x_c  # center along x
    ax.set_xlabel(r"$x$ ($\mu$m)")
    ax.set_ylabel(r"$v$ ($\mu$m/hr)")

    # drop labels where F is nan
    if bootstrap_flag:
        labels = labels[~np.isnan(F.ravel())]

    X, Y, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
    init_pts = clean_init_pts(init_pts, labels, nuke_red_in_middle)
    print(f"All unique colors: {init_pts.color.unique()}")

    for lbl, pts in init_pts.groupby("color"):
        if lbl == "gainsboro":
            continue

        if lbl == "red" and not include_red:
            continue

        _pts = pts[["x", "y"]].values

        if lbl != "wheat" or ignore_wheat:
            _pts = _pts[::n_skip]
            if mix_corn_salmon:
                lbl = "cornflowerblue"

        if lbl == "wheat":
            lbl = "goldenrod"
            if ignore_wheat:
                lbl = "cornflowerblue"

        if darker_shade:
            if lbl == "cornflowerblue":
                lbl = "royalblue"
            elif lbl == "salmon":
                lbl = "tomato"
            else:
                pass

        stream_kwargs = {"color": lbl, "start_points": _pts} | kwargs
        ax.streamplot(X, Y, Y, F, **stream_kwargs)
        ax.set_title(title)
        if zero_line:
            ax.axhline(y=0, lw=1, color="black", zorder=0)

        if show_terminal is not None:
            color = show_terminal.get("color", "black")
            s = show_terminal.get("size", 50)
            interior = show_terminal.get("interior", False)
            # mask = np.fabs(end_pts[:, 1]) < 10
            # end_pts = end_pts[mask]

            if interior:
                xy = (
                    pd.DataFrame({"x": end_pts[:, 0], "y": end_pts[:, 1]})
                    .query("x < 165 and x > 135 and y < 20 and y > -20")
                    .values
                )
                ax.scatter(
                    xy[:, 0] - x_c,
                    xy[:, 1],
                    s=s,
                    color=color,
                    marker="x",
                    zorder=10,
                )
            else:
                ax.scatter(
                    end_pts[:, 0] - x_c,
                    end_pts[:, 1],
                    s=s,
                    color=color,
                    marker="x",
                    zorder=10,
                )

    if septx:
        xmin, xmax, vmin, vmax, nbins = bounds
        _, _, init_pts = full_lattice(F, xmin, xmax, vmin, vmax, nbins)
        init_pts_df = clean_init_pts(init_pts, labels, nuke_red_in_middle)
        init_pts = init_pts_df[["x", "y"]].values
        labels = init_pts_df.color.values

        img = lattice_to_image(init_pts, labels, bounds)
        if not mix_corn_salmon:
            sepX_1 = get_separatrices(
                img, levels=[1.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            ax.plot(sepX_1[:, 0] - x_c, sepX_1[:, 1], lw=2, color="black")

        if np.any(np.where(img == 1)):
            arr = np.where(img == 1, 1, 0)  # cluster `wheat`
            sepX_2 = get_separatrices(
                arr, levels=[0.5], origin="lower", extent=[xmin, xmax, vmin, vmax]
            )
            ax.plot(sepX_2[:, 0] - x_c, sepX_2[:, 1], lw=2, color="black")


def process_gid(gid, dir_name):
    import pickle

    data = pickle.load(open(f"maps/{dir_name}/map_{gid}.pkl", "rb"))
    if len(data["labels"]) > 0:
        return data["F"], data["bounds"], np.array(data["labels"]), data["end_pts"]
    return data["F"], data["bounds"]


def sorted_glob(pathname):
    files = np.array(glob(pathname))
    return files[np.argsort([int(f.split("id")[-1].split(".")[0]) for f in files])]


def compute_msd(X):
    N = len(X)
    max_t = min(X, key=lambda x: len(x)).shape[0]
    res = []
    for arr in X:
        res.append(np.linalg.norm(arr[1:max_t] - arr[0], axis=1) ** 2)
    return np.arange(1, max_t, 1), np.array(res).sum(axis=0) / N

# Free Migration MSD

In [ ]:
gid = 151
file = f"/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/defaults/parquets/fulltake_gid{gid}.parquet"

grid_df = read_fulltake(file, scale_position=True)
xy_sets = grid_df.groupby("rid")[["x", "y"]]
xy_sets = [xy_sets.get_group(k).values for k in range(grid_df.rid.max())]

lag_times, msd = compute_msd(xy_sets)
_x = np.linspace(1.0, lag_times[-1], 700)
plt.figure(figsize=(4, 2), dpi=200)
plt.plot(lag_times, msd, color="black")
plt.plot(_x[:2], 5 * _x[:2] ** 2, label=r"$\sim t^2$", color="grey", ls="dotted")
plt.plot(_x[-400:-50], 25 * _x[-400:-50], label=r"$\sim t$", color="grey", ls="dashed")
plt.yscale("log")
plt.xscale("log")
plt.ylabel("MSD")
plt.xlabel("time [hr]")
plt.legend()

---

# New Geometries

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([1, 2, 0, 3, 4])
fig, hist_axs = plt.subplots(gids.size, 1, figsize=(3.5, 9), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

for k, (gid, hist_ax) in enumerate(zip(gids, hist_axs)):
    grid_df = read_fulltake(
        os.path.join(data_root, f"parquets/fulltake_gid{gid}.parquet"),
        scale_position=True,
    )
    xy = grid_df[["x", "y"]].values
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    chi_og = Substrate(200, 50, 0.01).mixed_two_state_sub(
        simbox_config["substrate"]["basin_dims"],
        simbox_config["substrate"]["bridge_dim"],
    )

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    f = 0.95
    chi = np.where(chi_og < f, 0.0, 1.0)
    y, x = np.where(chi < f)
    mask = np.where(x > 100)[0]
    chi[y[mask], x[mask]] = look[simbox_config["substrate"]["basin_dims"][1][0]]
    mask = np.where(x <= 100)[0]
    chi[y[mask], x[mask]] = look[simbox_config["substrate"]["basin_dims"][0][0]]
    # chi = np.where(chi == 2, np.nan, chi)

    hist_ax.hist(
        xy[:, 0] - 150,
        bins=50,
        density=True,
        color=plt.get_cmap("cool")((k + 1) / (gids.size + 1)),
    )
    hist_ax.axvline(0, color="black", linewidth=0.75)
    f = 1.1
    sub_ax = inset_axes(
        hist_ax,
        width=f,
        height=f,
        bbox_to_anchor=(1.04, 1.3),
        bbox_transform=hist_ax.transAxes,
        loc="upper right",
    )
    print(np.unique(chi))
    sub_ax.imshow(
        chi,
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        cmap="Greys_r",
        vmin=0.0,
        vmax=1.0,
    )
    sub_ax.contour(
        chi_og,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1],
    )
    sub_ax.set_xlim((90, 210))
    sub_ax.set_ylim((120, 180))
    sub_ax.set_axis_off()

hist_ax.set_xlabel(r"$x$ [$\mu$m]")
fig.text(-0.2, 0.5, "Occupancy", va="center", rotation="vertical")
plt.savefig("two_state_dim_sweeps.jpg", dpi=300)
plt.show()

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([1, 2, 0, 3, 4])
fig, axs = plt.subplots(gids.size, 1, figsize=(5, 15), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": True,
    # "color": "black"
}
nskip = 2

for k, (gid, ax) in enumerate(zip(gids[::2], axs[::2])):
    data = process_gid(gid, "diff_geoms")
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    chi_og = Substrate(200, 50, 0.01).mixed_two_state_sub(
        simbox_config["substrate"]["basin_dims"],
        simbox_config["substrate"]["bridge_dim"],
    )

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    f = 0.95
    chi = np.where(chi_og < f, 0.0, 1.0)
    y, x = np.where(chi < f)
    mask = np.where(x > 100)[0]
    chi[y[mask], x[mask]] = look[simbox_config["substrate"]["basin_dims"][1][0]]
    mask = np.where(x <= 100)[0]
    chi[y[mask], x[mask]] = look[simbox_config["substrate"]["basin_dims"][0][0]]
    # chi = np.where(chi == 2, np.nan, chi)

    heatmap(ax, *data, xlabel=True, cbar=True, cbar_label=True)

    streamplot(
        *data,
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        ignore_wheat=False,
        nuke_red_in_middle=False,
        include_red=False,
        show_terminal=None,
        darker_shade=True,
        **kwargs,
    )
    ax.set_xlim(np.array(ax.get_xlim()) + np.array([-1, 9]))
    if k != gids.size - 1:
        ax.set_xlabel("")

    f = 1.1
    sub_ax = inset_axes(
        ax,
        width=f,
        height=f,
        bbox_to_anchor=(0.95, 1.3),
        bbox_transform=ax.transAxes,
        loc="upper right",
    )
    sub_ax.imshow(
        chi,
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        cmap="Greys_r",
        vmin=0.0,
        vmax=1.0,
    )
    sub_ax.contour(
        chi_og,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1],
    )
    sub_ax.set_xlim((90, 210))
    sub_ax.set_ylim((120, 180))
    sub_ax.set_axis_off()

[ax.remove() for ax in axs if not ax.has_data()]
plt.savefig("two_state_dim_sweeps_streams.jpg", dpi=300)
plt.show()

---

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([5, 6])
fig, hist_axs = plt.subplots(gids.size, 1, figsize=(4, 3.5), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

for k, (gid, hist_ax) in enumerate(zip(gids, hist_axs)):
    grid_df = read_fulltake(
        os.path.join(data_root, f"parquets/fulltake_gid{gid}.parquet"),
        scale_position=True,
    )
    xy = grid_df[["x", "y"]].values
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    chi = Substrate(200, 50, 0.01).mixed_two_state_sub(
        simbox_config["substrate"]["basin_dims"],
        simbox_config["substrate"]["bridge_dim"],
    )

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    hist_ax.hist(
        xy[:, 0] - 150,
        bins=50,
        density=True,
        color=plt.get_cmap("cool")((k + 1) / (gids.size + 1)),
    )
    hist_ax.set_xlim((-75, 75))
    # hist_ax.set_ylim((0, 0.048))
    hist_ax.axvline(0, color="black", linewidth=0.75)
    f = 1.0
    sub_ax = inset_axes(
        hist_ax,
        width=f,
        height=f,
        bbox_to_anchor=(1.03, 1.25),
        bbox_transform=hist_ax.transAxes,
        loc="upper right",
    )
    sub_ax.contour(
        chi,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1.5],
    )
    sub_ax.set_xlim((75, 225))
    sub_ax.set_ylim((75, 225))
    sub_ax.set_axis_off()

hist_ax.set_xlabel(r"$x$ [$\mu$m]")
fig.text(-0.1, 0.5, "Occupancy", va="center", rotation="vertical")
plt.savefig("two_state_rectangular_basins.jpg", dpi=300)
plt.show()

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([5, 6])
fig, axs = plt.subplots(gids.size, 1, figsize=(5, 5), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": True,
    # "color": "black"
}
nskip = 2

for k, (gid, ax) in enumerate(zip(gids, axs)):
    data = process_gid(gid, "diff_geoms")
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    chi = Substrate(200, 50, 0.01).mixed_two_state_sub(
        simbox_config["substrate"]["basin_dims"],
        simbox_config["substrate"]["bridge_dim"],
    )

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    heatmap(ax, *data, xlabel=True, cbar=True, cbar_label=True)
    streamplot(
        *data,
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        ignore_wheat=False,
        nuke_red_in_middle=False,
        include_red=False,
        show_terminal=None,
        darker_shade=True,
        **kwargs,
    )
    ax.set_xlim(np.array(ax.get_xlim()) + np.array([-5, 25]))
    if k != gids.size - 1:
        ax.set_xlabel("")

    f = 1.1
    sub_ax = inset_axes(
        ax,
        width=f,
        height=f,
        bbox_to_anchor=(1.02, 1.2),
        bbox_transform=ax.transAxes,
        loc="upper right",
    )
    sub_ax.contour(
        chi,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1.5],
    )
    sub_ax.set_xlim((80, 220))
    sub_ax.set_ylim((80, 220))
    sub_ax.set_axis_off()

# plt.savefig("two_state_dim_sweeps_streams.jpg", dpi=300)
plt.show()

---

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([7, 8])
fig, hist_axs = plt.subplots(gids.size, 1, figsize=(4, 3.5), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

for k, (gid, hist_ax) in enumerate(zip(gids, hist_axs)):
    grid_df = read_fulltake(
        os.path.join(data_root, f"parquets/fulltake_gid{gid}.parquet"),
        scale_position=True,
    )
    xy = grid_df[["x", "y"]].values
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    if simbox_config["substrate"]["kind"] == "square-triangle":
        chi_og = Substrate(200, 50, 0.01).sq_tri_two_state_sub()
    else:
        chi_og = Substrate(200, 50, 0.01).sq_rhom_two_state_sub()

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    hist_ax.hist(
        xy[:, 0] - 150,
        bins=50,
        density=True,
        color=plt.get_cmap("cool")((k + 1) / (gids.size + 1)),
    )
    hist_ax.set_xlim((-70, 70))
    hist_ax.set_ylim((0, 0.048))
    hist_ax.axvline(0, color="black", linewidth=0.75)
    f = 1.0
    sub_ax = inset_axes(
        hist_ax,
        width=f,
        height=f,
        bbox_to_anchor=(1.04, 1.25),
        bbox_transform=hist_ax.transAxes,
        loc="upper right",
    )
    sub_ax.contour(
        chi_og,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1.5],
    )
    sub_ax.set_xlim((80, 220))
    sub_ax.set_ylim((80, 220))
    sub_ax.set_axis_off()

hist_ax.set_xlabel(r"$x$ [$\mu$m]")
fig.text(-0.1, 0.5, "Occupancy", va="center", rotation="vertical")
plt.savefig("square_rhom_tri_states.jpg", dpi=300)
plt.show()

In [ ]:
from substrate.substrates import Substrate
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/diff_geoms/"
gids = np.array([7, 8])
fig, axs = plt.subplots(gids.size, 1, figsize=(5, 5), sharex=True, sharey=True)
cmap = plt.get_cmap(name="Purples")
look = {30: 1, 34: 0.75, 38: 0.5, 42: 0}

kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": True,
    # "color": "black"
}
nskip = 2

for k, (gid, ax) in enumerate(zip(gids, axs)):
    data = process_gid(gid, "diff_geoms")
    simbox_config = yaml.safe_load(
        open(os.path.join(data_root, f"configs/grid_id{gid}/simbox.yaml"), "rb")
    )
    if simbox_config["substrate"]["kind"] == "square-triangle":
        chi = Substrate(200, 50, 0.01).sq_tri_two_state_sub()
    else:
        chi = Substrate(200, 50, 0.01).sq_rhom_two_state_sub()

    print("# of runs:", grid_df.rid.unique().size)
    print(
        f"{simbox_config['substrate']['basin_dims']}"
        + " x "
        + f"{simbox_config['substrate']['bridge_dim']}"
    )

    heatmap(ax, *data, xlabel=True, cbar=True, cbar_label=True)
    streamplot(
        *data,
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=True,
        ignore_wheat=False,
        nuke_red_in_middle=False,
        include_red=False,
        show_terminal=None,
        darker_shade=True,
        **kwargs,
    )
    ax.set_xlim(np.array(ax.get_xlim()) + np.array([-5, 25]))
    if k != gids.size - 1:
        ax.set_xlabel("")

    f = 1.1
    sub_ax = inset_axes(
        ax,
        width=f,
        height=f,
        bbox_to_anchor=(1.02, 1.2),
        bbox_transform=ax.transAxes,
        loc="upper right",
    )
    sub_ax.contour(
        chi,
        levels=[0.5],
        origin="lower",
        extent=[0, 50 * 6, 0, 50 * 6],
        colors=["black"],
        linewidths=[1.5],
    )
    sub_ax.set_xlim((80, 220))
    sub_ax.set_ylim((80, 220))
    sub_ax.set_axis_off()

plt.savefig("square_rhom_tri_states_streams.jpg", dpi=300)
plt.show()

## Toying with triangular ratchet

In [ ]:
from importlib import reload
from substrate import substrates

reload(substrates)

from substrate.substrates import Substrate

chi = Substrate(200, 50, 0.2).tri_ratchets()

plt.figure(dpi=200, figsize=(5, 4))
plt.imshow(chi, origin="lower", extent=[0, 50 * 6, 0, 50 * 6], cmap="Greys_r")
plt.colorbar()

xs = [
    185.61340206,
    110.61340206,
    73.11340206,
    35.61340206,
    149.3814433,
    260.61340206,
    223.11340206,
]
ys = [149.8] * len(xs)
plt.scatter(xs, ys)
plt.xlabel("x [mu]")
plt.ylabel("y [mu]")
plt.ylim((110, 190))
plt.show()

# Sweeping over $\tau_f$ and $\tau$ 

In [ ]:
data_root = "/mnt/c/Users/pedro/OneDrive - Johns Hopkins/Documents/Research/hopper/_server/sim_data/tau_sweeps/"
configs = sorted_glob(os.path.join(data_root, "configs/*"))
parquets = sorted_glob(os.path.join(data_root, "parquets/*"))
for config, parq in zip(configs, parquets):
    simbox_yaml = yaml.safe_load(open(os.path.join(config, "simbox.yaml"), "rb"))
    cell_yaml = yaml.safe_load(open(os.path.join(config, "cell.yaml"), "rb"))
    grid_df = read_fulltake(parq, scale_position=True)
    print(
        f"Substrate: {simbox_yaml['substrate']['kind']}\ttau_f: {cell_yaml['tau_mvg']}\ttau: {cell_yaml['tau']}\ttotal # of runs: {grid_df.rid.unique().size}"
    )

In [ ]:
nrows, ncols = 5, 5
fig, axs = plt.subplots(
    nrows, ncols, figsize=(3 * ncols, 3 * nrows), sharex=True, sharey=True
)

# 0-24 -- two-state
# 25 - 49 -- rectangular
gids = range(25)
kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 0.75,
    "integration_direction": "forward",
    "broken_streamlines": True,
}
nskip = 2
inline = False
render = lambda f, dpi=300: plt.show() if inline else plt.savefig(f"{f}.jpg", dpi=dpi)

bistable = [0, 1, 2, 3, 4, 7, 8, 9, 14, 15, 20]
midpts = [1, 5, 7, 10, 14, 15, 20]
# streams
for ax, gid in zip(axs.flatten(), gids):
    limitcycle = True
    ignore_wheat = True
    include_red = True
    show_terminal = None

    if gid in bistable:
        limitcycle = False
        show_terminal = {"color": "black", "size": 20}

    if gid in midpts:
        ignore_wheat = False

    if gid == 7:
        include_red = False

    streamplot(
        *process_gid(gid, "tau_sweeps"),
        ax,
        n_skip=nskip,
        zero_line=True,
        mix_corn_salmon=limitcycle,
        ignore_wheat=ignore_wheat,
        nuke_red_in_middle=False,
        include_red=include_red,
        show_terminal=show_terminal,
        **kwargs,
    )
    ax.set_ylabel("")
    ax.set_xlabel("")
    ax.set_ylim((-300, 300))
    ax.set_xlim((-50, 50))
    if gid in [0, 5, 10, 15, 20]:
        ax.set_ylabel(r"$v$ ($\mu$m/hr)")
    if gid in [20, 21, 22, 23, 24]:
        ax.set_xlabel(r"$x$ ($\mu$m)")

taus = np.array([1, 2, 4, 6, 8])
tau_fs = np.array([1.0, 2.2, 3.6, 5.1, 7.0])
title_tf = lambda x: r"$\tau_{f}=$" + f"{x} " + r"$\mathrm{s}$"
title_t = lambda x: r"$\tau=$" + f"{x} " + r"$\mathrm{min}$"

[
    ax.set_title(title_tf(tf), fontsize=26, y=1.1)
    for ax, tf in zip(axs.flatten()[:5], tau_fs)
]
[
    ax.text(
        ax.get_xlim()[0] - 100,
        np.array(ax.get_ylim()).mean() - 150,
        title_t(t),
        rotation=90,
        fontdict={"fontsize": 26},
    )
    for ax, t in zip(axs[:, 0], taus)
]

plt.tight_layout()
plt.savefig("two_state_taus_sweep.jpg", dpi=300)
plt.show()

# Default Cell in Larger Basin

In [ ]:
kwargs = {
    "density": 10,
    "linewidth": 0.5,
    "arrowsize": 1.0,
    "integration_direction": "forward",
    "broken_streamlines": True,
}
nskip = 2

fig, axs = plt.subplots(1, 2, figsize=(6, 3), dpi=300, sharex=True, sharey=True)
[
    streamplot(
        *process_gid(gid, ""),
        ax,
        mix_corn_salmon=mix,
        include_red=False,
        title=lbl,
        show_terminal=st,
        septx=septx,
        **kwargs
    )
    for gid, mix, lbl, st, septx, ax in zip(
        [148, 147],
        [True, False],
        [r"$41.8\mu \mathrm{m} \times 41.8 \mu \mathrm{m}$", r"$45.6 \mu \mathrm{m} \times 45.6 \mu \mathrm{m}$"],
        [None, {"color": "black", "size": 20}],
        [False, True],
        axs,
    )
]
plt.tight_layout()
plt.savefig("default_cell_larger_basins.jpg")
plt.show()